#Price forecasting with DT 3 algorithm and displaying buy and sell signals with indicator Moving average

In [ ]:
# Import necessary libraries
import yfinance as yf  # Used to download stock data
import numpy as np  # For numerical operations
from sklearn.tree import DecisionTreeClassifier  # Decision tree classifier for making buy/sell decisions
import matplotlib.pyplot as plt  # For creating plots
from sklearn.metrics import confusion_matrix, classification_report  # Used for evaluating model performance

# Define the stock symbol, start and end date for data retrieval
ticker = "TSLA"  # Stock symbol for Tesla
start_date = "2022-06-30"
end_date = "2023-06-30"

# Download historical stock data for the specified period
data = yf.download(ticker, start=start_date, end=end_date)

# Calculate daily returns and shift the data by one day
data['Return'] = data['Close'].pct_change().shift(-1)

# Remove rows with missing values (NaN)
data.dropna(inplace=True)

# Prepare the data for machine learning
X = np.array(data['Return']).reshape(-1, 1)  # Input features (returns)
y = np.where(data['Return'].shift(-1) > 0, 1, 0)  # Output labels (1 for buy, 0 for sell)

# Split the data into training and testing sets
split = int(0.8 * len(data))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Create a decision tree classifier and fit it to the training data
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Initialize lists to track buy and sell dates, prices, and calculate shares
buy_dates = []
sell_dates = []
buy_prices = []
sell_prices = []
shares = 0

# Determine buy and sell points based on the model predictions
for i in range(len(y_pred)):
    if y_pred[i] == 1:
        if shares == 0:
            buy_dates.append(data.index[i + split])
            buy_prices.append(data['Close'][i + split])
        shares += 10000 / data['Close'][i + split]
    else:
        if shares > 0:
            sell_dates.append(data.index[i + split])
            sell_prices.append(data['Close'][i + split])
        shares = 0

# Create a plot showing stock price, moving averages, buy, and sell points
plt.figure(figsize=(10, 6))
plt.plot(data.index[split:], data['Close'][split:], label='Price')
rolling_mean = data['Close'].rolling(window=20).mean()
plt.plot(data.index[split:], rolling_mean[split:], label='Moving Average (20 days)')
rolling_mean = data['Close'].rolling(window=50).mean()
plt.plot(data.index[split:], rolling_mean[split:], label='Moving Average (50 days)')
plt.plot(buy_dates, buy_prices, 'g^', markersize=10, label='Buy')
plt.plot(sell_dates, sell_prices, 'rv', markersize=10, label='Sell')
plt.legend()
plt.title(f"{ticker} Stock Trading Decisions")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()

# Create a plot showing actual and predicted stock price movements
plt.figure(figsize=(10, 6))
plt.plot(data.index[split:], y_test, label='Actual')
plt.plot(data.index[split:], y_pred, label='Predicted')
plt.legend()
plt.title(f"{ticker} Stock Price Prediction")
plt.xlabel("Date")
plt.ylabel("Prediction")
plt.show()


* This code retrieves stock data, builds a decision tree model to make buy/sell decisions based on returns, and visualizes the results with price charts. The code also uses rolling averages to analyze price trends.